OneData Assessment 

Task 1: Upload Data to MinIO

In [ ]:
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError


s3 = boto3.client(
    's3',
    endpoint_url='http://127.0.0.1:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    config=Config(signature_version='s3v4'),
    region_name='us-east-1'
)

bucket_name = "my-bucket"


try:
    s3.head_bucket(Bucket=bucket_name)
    print("✅ Bucket already exists")
except ClientError:
    s3.create_bucket(Bucket=bucket_name)
    print("✅ Bucket created")


files = [
    r"E:\Assessment_Data\care_reception.csv",
    r"E:\Assessment_Data\event.csv",
    r"E:\Assessment_Data\participation.csv",
    r"E:\Assessment_Data\patients.csv"
]


for file_path in files:
    file_name = file_path.split("\\")[-1]  
    object_name = f"raw/assessment/{file_name}"
    s3.upload_file(file_path, bucket_name, object_name)
    print(f"✅ Uploaded {file_name} to {object_name}")


Task 2: Analyze and Map All Data into One Table

In [5]:
import pandas as pd

In [6]:

patients = pd.read_csv("E:/Assessment_Data/patients.csv")
care_reception = pd.read_csv("E:/Assessment_Data/care_reception.csv")
event = pd.read_csv("E:/Assessment_Data/event.csv")
participation = pd.read_csv("E:/Assessment_Data/participation.csv")


In [7]:
print(patients.head(10))

   Unnamed: 0   id first_name  last_name  gender  organization_id  \
0           0  145        Raj      kumar    MALE                1   
1           1  146      Barry   Martinez    MALE                1   
2           2  147     Janice  Castaneda    MALE                1   
3           3  148    Rebecca       Tran  FEMALE                1   
4           4  149     Jeremy       Ward    MALE                1   
5           5  150     Justin     Foster     NaN                1   
6           6  151     Dinesh      kumar    MALE                1   
7           7  152     Steven     Guerra     NaN                1   
8           8  153   Kimberly      Smith  FEMALE                1   
9           9  154    Valerie     Larson  FEMALE                1   

                         created_at  created_by  status_id  language_id  \
0  2024-10-21 11:13:19.200878+00:00         139          2            1   
1  2024-10-21 11:44:27.982926+00:00         139          2            1   
2  2024-10-21 1

In [8]:
patients['Full_Name'] = patients['first_name'] + ' ' + patients['last_name']


In [9]:
patients = patients[[
    'id', 'first_name', 'last_name', 'gender', 'date_of_birth', 
    'email', 'phone', 'marital_status', 'Full_Name'
]]

In [ ]:
patients=patients.duplicated()

In [10]:
print(patients)

        id first_name   last_name  gender date_of_birth  \
0      145        Raj       kumar    MALE    1987-02-02   
1      146      Barry    Martinez    MALE    1990-01-10   
2      147     Janice   Castaneda    MALE    1998-07-16   
3      148    Rebecca        Tran  FEMALE    2000-02-23   
4      149     Jeremy        Ward    MALE    1990-01-02   
...    ...        ...         ...     ...           ...   
3550  2554        vbn         vnb     NaN           NaN   
3551  2559      Selva          ss     NaN           NaN   
3552  2564      Sudha          MA  FEMALE    1996-12-22   
3553  2566      Nuzha  MeeraLebbe     NaN    1999-11-20   
3554  2567      Anaas     Mohamed    MALE    1998-11-20   

                       email           phone  marital_status         Full_Name  
0                        NaN   91 9894336789             1.0         Raj kumar  
1                        NaN   91 9876543210             1.0    Barry Martinez  
2                        NaN  +91 9876566815    

In [11]:
patients['email'] = patients['email'].fillna('Unknown')
patients['phone'] = patients['phone'].fillna('Unknown')
patients['gender'] = patients['gender'].fillna('Unknown')


In [12]:
print(patients)

        id first_name   last_name   gender date_of_birth  \
0      145        Raj       kumar     MALE    1987-02-02   
1      146      Barry    Martinez     MALE    1990-01-10   
2      147     Janice   Castaneda     MALE    1998-07-16   
3      148    Rebecca        Tran   FEMALE    2000-02-23   
4      149     Jeremy        Ward     MALE    1990-01-02   
...    ...        ...         ...      ...           ...   
3550  2554        vbn         vnb  Unknown           NaN   
3551  2559      Selva          ss  Unknown           NaN   
3552  2564      Sudha          MA   FEMALE    1996-12-22   
3553  2566      Nuzha  MeeraLebbe  Unknown    1999-11-20   
3554  2567      Anaas     Mohamed     MALE    1998-11-20   

                       email           phone  marital_status         Full_Name  
0                    Unknown   91 9894336789             1.0         Raj kumar  
1                    Unknown   91 9876543210             1.0    Barry Martinez  
2                    Unknown  +91 98

In [13]:
print(care_reception.head(10))

   participation_id  patient_id  unregistered_user_id  address_id message  \
0               578        38.0                   NaN         NaN     NaN   
1                72        37.0                   NaN         NaN     NaN   
2               509        56.0                   NaN         NaN     NaN   
3                73        37.0                   NaN         NaN     NaN   
4                74        37.0                   NaN         NaN     NaN   
5                75        38.0                   NaN         NaN     NaN   
6               512        41.0                   NaN         NaN     NaN   
7                76        38.0                   NaN         NaN     NaN   
8                95        38.0                   NaN         NaN     NaN   
9               100        42.0                   NaN         NaN     NaN   

   status  meeting_status patient_attendant_name  \
0       1           False                    NaN   
1       3           False                    NaN

In [14]:
care_reception = care_reception[['participation_id', 
    'patient_id', 
    'unregistered_user_id', 
    'status', 
    'meeting_status']]

In [15]:
print(care_reception.head(10))

   participation_id  patient_id  unregistered_user_id  status  meeting_status
0               578        38.0                   NaN       1           False
1                72        37.0                   NaN       3           False
2               509        56.0                   NaN       1           False
3                73        37.0                   NaN       3           False
4                74        37.0                   NaN       3           False
5                75        38.0                   NaN       1           False
6               512        41.0                   NaN       1           False
7                76        38.0                   NaN       3           False
8                95        38.0                   NaN       3           False
9               100        42.0                   NaN       1           False


In [16]:
print(participation.head(10))

     id  event_id  created_by                        created_at  updated_by  \
0  2688      2154         139  2024-10-22 12:24:03.052994+00:00         NaN   
1  1609      1257          33  2024-10-21 04:28:27.377719+00:00         NaN   
2  1552      1200          33  2024-10-21 04:27:40.261247+00:00         NaN   
3  1553      1201          33  2024-10-21 04:27:40.400597+00:00         NaN   
4  1554      1202          33  2024-10-21 04:27:40.433044+00:00         NaN   
5  1555      1203          33  2024-10-21 04:27:40.481896+00:00         NaN   
6  1556      1204          33  2024-10-21 04:27:40.545213+00:00         NaN   
7  1557      1205          33  2024-10-21 04:27:40.680962+00:00         NaN   
8  1558      1206          33  2024-10-21 04:27:40.721584+00:00         NaN   
9  1559      1207          33  2024-10-21 04:27:40.772617+00:00         NaN   

  updated_at  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  
5        NaN  
6        NaN  
7        N

In [17]:
participation = participation[[  'id', 'event_id', 'created_by', 'created_at'
]]

In [18]:
print(participation.head(10))

     id  event_id  created_by                        created_at
0  2688      2154         139  2024-10-22 12:24:03.052994+00:00
1  1609      1257          33  2024-10-21 04:28:27.377719+00:00
2  1552      1200          33  2024-10-21 04:27:40.261247+00:00
3  1553      1201          33  2024-10-21 04:27:40.400597+00:00
4  1554      1202          33  2024-10-21 04:27:40.433044+00:00
5  1555      1203          33  2024-10-21 04:27:40.481896+00:00
6  1556      1204          33  2024-10-21 04:27:40.545213+00:00
7  1557      1205          33  2024-10-21 04:27:40.680962+00:00
8  1558      1206          33  2024-10-21 04:27:40.721584+00:00
9  1559      1207          33  2024-10-21 04:27:40.772617+00:00


In [19]:
print(event.head(10))

     id                    title                             start  \
0  1286                  WALKINS  2024-10-21 05:53:32.449000+00:00   
1  1269                     Test         2024-10-21 05:00:00+00:00   
2  1287                  WALKINS  2024-10-21 06:02:20.519000+00:00   
3  1307  Video call consultation         2024-10-21 09:00:00+00:00   
4  1308  Video call consultation         2024-10-21 09:10:00+00:00   
5  1309  Video call consultation         2024-10-21 09:20:00+00:00   
6  1310  Video call consultation         2024-10-21 09:30:00+00:00   
7  1311  Video call consultation         2024-10-21 09:40:00+00:00   
8  1312  Video call consultation         2024-10-21 09:50:00+00:00   
9  1318                     Test         2024-10-21 08:00:00+00:00   

                                end                      until  \
0  2024-10-21 06:22:32.449000+00:00                        NaN   
1         2024-10-21 05:15:00+00:00                        NaN   
2  2024-10-21 06:32:20.519000+0

In [20]:

event.columns = event.columns.str.strip()


In [21]:
event = event[['id', 'title', 'start', 'end', 'until']]

In [22]:
event['until']= event['until'].fillna('unknown')

In [23]:
event['start'] = pd.to_datetime(event['start'], utc=True, errors='coerce')
event['end'] = pd.to_datetime(event['end'], utc=True, errors='coerce')
event['until'] = pd.to_datetime(event['until'], utc=True, errors='coerce')


C:\Users\Saravanan\AppData\Local\Temp\ipykernel_20452\3435083340.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  event['until'] = pd.to_datetime(event['until'], utc=True, errors='coerce')


In [24]:
print(event.head(10))

     id                    title                            start  \
0  1286                  WALKINS 2024-10-21 05:53:32.449000+00:00   
1  1269                     Test                              NaT   
2  1287                  WALKINS 2024-10-21 06:02:20.519000+00:00   
3  1307  Video call consultation                              NaT   
4  1308  Video call consultation                              NaT   
5  1309  Video call consultation                              NaT   
6  1310  Video call consultation                              NaT   
7  1311  Video call consultation                              NaT   
8  1312  Video call consultation                              NaT   
9  1318                     Test                              NaT   

                               end                     until  
0 2024-10-21 06:22:32.449000+00:00                       NaT  
1                              NaT                       NaT  
2 2024-10-21 06:32:20.519000+00:00                 

In [25]:
care_reception.columns = care_reception.columns.str.strip()
patients.columns = patients.columns.str.strip()
participation.columns = participation.columns.str.strip()
event.columns = event.columns.str.strip()


In [26]:
print("Care Reception columns after strip:", care_reception.columns)


Care Reception columns after strip: Index(['participation_id', 'patient_id', 'unregistered_user_id', 'status',
       'meeting_status'],
      dtype='object')


Merging tables

In [27]:
unified_df = patients.copy()

In [28]:
unified_df = patients.merge(
    care_reception[['patient_id', 'participation_id', 'status', 'meeting_status']],
    left_on='id',  
    right_on='patient_id',
    how='left'
)

In [29]:
unified_df = unified_df.merge(participation[['id', 'event_id']], 
                              left_on='participation_id', 
                              right_on='id', 
                              how='left')



In [30]:
unified_df = unified_df.merge(
    participation[['id', 'event_id']],
    left_on='participation_id',
    right_on='id',
    how='left',
    suffixes=('', '_participation')
)


In [31]:
print(unified_df)

       id_x first_name   last_name   gender date_of_birth  \
0       145        Raj       kumar     MALE    1987-02-02   
1       145        Raj       kumar     MALE    1987-02-02   
2       145        Raj       kumar     MALE    1987-02-02   
3       145        Raj       kumar     MALE    1987-02-02   
4       145        Raj       kumar     MALE    1987-02-02   
...     ...        ...         ...      ...           ...   
13772  2564      Sudha          MA   FEMALE    1996-12-22   
13773  2566      Nuzha  MeeraLebbe  Unknown    1999-11-20   
13774  2567      Anaas     Mohamed     MALE    1998-11-20   
13775  2567      Anaas     Mohamed     MALE    1998-11-20   
13776  2567      Anaas     Mohamed     MALE    1998-11-20   

                        email          phone  marital_status  \
0                     Unknown  91 9894336789             1.0   
1                     Unknown  91 9894336789             1.0   
2                     Unknown  91 9894336789             1.0   
3          

In [32]:
print(unified_df.columns.tolist())


['id_x', 'first_name', 'last_name', 'gender', 'date_of_birth', 'email', 'phone', 'marital_status', 'Full_Name', 'patient_id', 'participation_id', 'status', 'meeting_status', 'id_y', 'event_id', 'id', 'event_id_participation']


In [33]:
columns_order = ['patient_id', 'Full_Name', 'date_of_birth', 'gender', 'phone', 
                 'status', 'meeting_status', 'participation_id', 'event_id']

unified_df = unified_df[columns_order]


In [34]:
print(unified_df.head())
print("Total rows:", unified_df.shape[0])


   patient_id  Full_Name date_of_birth gender          phone  status  \
0       145.0  Raj kumar    1987-02-02   MALE  91 9894336789     1.0   
1       145.0  Raj kumar    1987-02-02   MALE  91 9894336789     1.0   
2       145.0  Raj kumar    1987-02-02   MALE  91 9894336789     1.0   
3       145.0  Raj kumar    1987-02-02   MALE  91 9894336789     1.0   
4       145.0  Raj kumar    1987-02-02   MALE  91 9894336789     1.0   

  meeting_status  participation_id  event_id  
0          False            4044.0       NaN  
1          False            6906.0    5426.0  
2          False            7018.0    5542.0  
3          False            7986.0    6526.0  
4          False           12027.0   10423.0  
Total rows: 13777


In [38]:
unified_df= unified_df.duplicated()
unified_df= unified_df.drop_duplicates()
print("Duplicates removed. Total rows now:", unified_df.shape[0])   

Duplicates removed. Total rows now: 2


In [39]:
unified_df.to_csv("E:/Assessment_Data/unified_table.csv", index=False)
print("✅ Unified table saved successfully!")


✅ Unified table saved successfully!


In [40]:
bucket_name = "my-bucket"  
file_path = "E:/Assessment_Data/unified_table.csv"
object_name = "raw/new/unified_table.csv"


In [41]:
try:
    s3.upload_file(file_path, bucket_name, object_name)
    print("✅ File uploaded successfully to MinIO!")
except Exception as e:
    print("❌ Upload failed:", e)


✅ File uploaded successfully to MinIO!
